# GEEO Tutorial 1 - Spatial Tiling and Metadata

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/leonsnill/geeo/blob/master/docs/tutorial_1_spatial-tiling-and-metadata.ipynb)

In [2]:
import ee
ee.Authenticate()
ee.Initialize(project='eexnill')

import geeo

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


Google Earth Engine hides much of the underlying complexities for mass-processing geospatial data, including spatial data organization in general, and proejctions in particular. *"[Earth Engine is designed so that you rarely have to worry about map projections when doing computations.](https://developers.google.com/earth-engine/guides/projections)"*. This is comfortable, but also bears its risks. In case you are coding in GEE yourself, we highly recommend getting familiar with the concepts of [scale](https://developers.google.com/earth-engine/guides/scale), [projections](https://developers.google.com/earth-engine/guides/projections), and [resampling](https://developers.google.com/earth-engine/guides/resample). **Geeo does potentially necessary spatial metadata handling for you within its core processing routine.**  

Although there is generally no need in GEE to specify a projection for instructing computations (with a few exceptions), and many of the spatial metadata and processing can simply be inferred or set to default settings (e.g. EPSG:4326 for crs), it is generally good practise to be explicit about these settings and being aware of the influence they can have. 

### Spatial Metadata

**When triggering an export, the following settings should ideally be explicitly set:**

- The **pixel resolution** (termed ['scale'](https://developers.google.com/earth-engine/guides/scale) in GEE):
- The **Coordinate Reference System (CRS)** or 'projection'
- The **resampling method**: By default, Nearest Neighbour (NN) resampling is used if neither 'bilinear' or 'bicubic' are specified.
- The **Region Of Interest (ROI)** or spatial extent/bounding box of the output image

In geeo, these are either set in the parameter .yml-file / dictionary (`PIX_RES`, `CRS`, `RESAMPLING_METHOD`, `ROI`) or when using the auxiliary function `export_img()`.

Optionally, the user can also precisely define the affine transform (`CRS_TRANSFORM`) to use for the export, as well as the image dimensions (`IMG_DIMENSIONS`). Note from the GEE docs: *"to get a block of pixels precisely aligned to another data source, specify dimensions, crs and crsTransform."*. Geeo can handle precisely aligning the pixels onto a desired grid based on the `ROI` (more on this below).

### Spatial Tiling
**Geeo has in-built functions for creating a spatial tiling scheme for a given CRS.** This faciliates spatial organization of the data similar to the datacube concept and allows precisely aligning output images onto a grid scheme. Especially when working with long and/or dense time series, as well as when working with different geospatial raster datasets in a given region, spatial tiling schemes can be extremely usefull.

- **Grid:** The spatial, regular division of the space into tiles starting the grid origin.
- **Grid Origin:** Upper left corner coordinate where numbering of tiles start in X and Y direction
- **Tile:** A (square) grid cell covering an area defined by the tile size. Associated with a unique identifier (X000_Y0000).
- **Tile Size:** The n times n size in meters covered by a tile.


<br>

<div>
<img src="https://raw.githubusercontent.com/leonsnill/geeo/master/geeo/data/fig/datacube.svg" width="50%" style="display:block; margin: 0 auto;"/>
</div>

In geeo, the user can use the functions `create_tiles()` and `create_glance_tiles()` to construct a tiling scheme for a given CRS, area, and tile size.

----

## User-defined CRS tiling scheme

Optionally, the user can specify a output directory to which the GeoDataFrame will be saved as GeoPackage (.gpkg) file. The user can futher specify a vector file to clip the resulting grid to and wether to filter the tiles to terrestrial surfaces (i.e., masking oceans and large lakes).

In [2]:
?geeo.create_tiles

Signature:
geeo.create_tiles(
    crs,
    extent=None,
    tile_size=150000,
    origin=None,
    vector_roi=None,
    output_dir=None,
    land_mask=False,
    land_mask_path=None,
)
Docstring: <no docstring>
File:      c:\users\leonx\documents\repos\geeo\geeo\misc\spacetime.py
Type:      function

The user defines the CRS, spatial extent and tile size. The spatial extent can be specied by using a rectangular extent, a origin coordinates, or a vector ROI. Optionally, the user can specify a output directory to which the GeoDataFrame will be saved as GeoPackage (.gpkg) file. The user can futher specify a vector file to clip the resulting grid to and wether to filter the tiles to terrestrial surfaces (i.e., masking oceans and large lakes).

To create a 30x30km grid for Germany:

In [3]:
import geemap
import geopandas as gpd

gdf_countries = gpd.read_file('https://raw.githubusercontent.com/leonsnill/geeo/master/geeo/data/ne_50m_admin_countries.gpkg')
germany = gdf_countries[gdf_countries['ADMIN'] == 'Germany']

M = geemap.Map(center=[51, 10], zoom=6)
M.add_basemap('HYBRID')
M.add_gdf(germany, style={'color': 'blue'}, layer_name="Germany", zoom_to_layer=False)
M

Map(center=[51, 10], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', tran…

In [6]:
germany_tiles = geeo.create_tiles(crs='EPSG:3035', vector_roi=germany, tile_size=30000, land_mask=True)
germany_tiles.head()

,geometry,ID,X,Y
0,"POLYGON ((4210570.037 3521502.597, 4210570.037...",X006-Y000,6,0
1,"POLYGON ((4240570.037 3521502.597, 4240570.037...",X007-Y000,7,0
2,"POLYGON ((4270570.037 3521502.597, 4270570.037...",X008-Y000,8,0
3,"POLYGON ((4300570.037 3521502.597, 4300570.037...",X009-Y000,9,0
4,"POLYGON ((4180570.037 3491502.597, 4180570.037...",X005-Y001,5,1


In [7]:
M = geemap.Map(center=[51, 10], zoom=6)
M.add_basemap('HYBRID')
M.add_gdf(germany, style={'color': 'blue'}, layer_name="Germany", zoom_to_layer=False)
M.add_gdf(germany_tiles, style={'color': 'red'}, layer_name="Germany Tiles", zoom_to_layer=False)
M

Map(center=[51, 10], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', tran…

The X and Y position (e.g. `X003-Y001`) creates a unique and spatially explicit identifier.  

----

## Modified Global LANd Cover mapping and Estimation (GLANCE) Grids

**Geeo contains a modified version of the Global LANd Cover mapping and Estimation (GLANCE) projection and tiling system** (https://github.com/measures-glance/glance-grids). 
GLANCE grids is a tile gridding system that uses Lambert Azimuthal Equal Area projections for different 'continents' to minimize distortion for each region. Originally, the grid system is based on a 30m pixel and with tiles that are 5000x5000 pixels in size, i.e. 150x150km tiles. Geeo allows to specify other tile sizes that all cover the common pixel sizes of medium resolution satellite data (10m, 20m, 30m). 

<br>

<div style="text-align:center;">
    <img src="https://raw.githubusercontent.com/leonsnill/geeo/master/geeo/data/GLANCE-tiles/glance_continents_overview.png" width="50%"/>
    <p><em>Figure: GLANCE grids overview (source: https://measures-glance.github.io/glance-grids/)</em></p>
</div>

Use `create_glance_tiles()` to create a new grid for a continent.
The parameters of the function are:

In [6]:
?geeo.create_glance_tiles

Signature:
geeo.create_glance_tiles(
    continent_code,
    tile_size=150000,
    vector_roi=None,
    output_dir=None,
    zone_mask=False,
    land_mask=False,
)
Docstring:
Create grid GeoPackage files based on continent code and grid parameters with explicit ID naming.
Optionally restrict the grid to land surfaces using a land mask.
Parameters:
- continent_code (str): The code of the continent for which to create the grid. Either AF, AN, AS, EU, OC, NA, SA or use "ALL" for all continents.
- tile_size (int): The size of the main grid tile in meters. Must be one of [1200000, 600000, 300000, 150000, 75000, 30000].
- vector_roi (str or GeoDataFrame): The shapefile path or GeoDataFrame to clip the grid with. Default is None.
- output_dir (str): The directory to save the grid GeoPackage files. Default is None.
- land_mask (bool): Whether to restrict the grid to land surfaces. Default is False (no masking).
Returns:
- GeoDataFrame: A GeoDataFrame containing the grid tiles with IDs for all

The user defines the continent (AF, AN, AS, EU, OC, NA, SA) and desired tile size (1200000, 600000, 300000, 150000, 75000, 30000 km). Optionally, the user can specify a output directory to which the GeoDataFrame will be saved as GeoPackage (.gpkg) file. The user can futher specify a vector file to clip the resulting grid to and wether to filter the tiles to terrestrial surfaces (i.e., masking oceans and large lakes).

To create a 150x150km grid for European landmasses:

In [7]:
glance_eu = geeo.create_glance_tiles(continent_code='EU', tile_size=150000, land_mask=True)

Filtering grid tiles using the land mask...


In [8]:
import geemap

M = geemap.Map(center=[49, 12], zoom=3)
M.add_basemap('HYBRID')
M.add_gdf(glance_eu, style={'color': 'red'}, layer_name="EU Glance", zoom_to_layer=False)
M

Map(center=[49, 12], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', tran…

**The resulting grid contains multiple attribute columns that encode the spatial X and Y position of the tile:**

In [9]:
glance_eu.head()

,geometry,ID1200,ID600,ID300,ID150,X1200,X600,X300,X150,Y1200,Y600,Y300,Y150
197,"POLYGON ((-255560 2746245, -255560 2896245, -1...",EU_1200-X004-Y000,EU_600-X008-Y000,EU_300-X017-Y001,EU_150-X035-Y003,4,8,17,35,0,0,1,3
198,"POLYGON ((-105560 2746245, -105560 2896245, 44...",EU_1200-X004-Y000,EU_600-X009-Y000,EU_300-X018-Y001,EU_150-X036-Y003,4,9,18,36,0,0,1,3
199,"POLYGON ((44440 2746245, 44440 2896245, 194440...",EU_1200-X004-Y000,EU_600-X009-Y000,EU_300-X018-Y001,EU_150-X037-Y003,4,9,18,37,0,0,1,3
200,"POLYGON ((194440 2746245, 194440 2896245, 3444...",EU_1200-X004-Y000,EU_600-X009-Y000,EU_300-X019-Y001,EU_150-X038-Y003,4,9,19,38,0,0,1,3
251,"POLYGON ((-255560 2596245, -255560 2746245, -1...",EU_1200-X004-Y000,EU_600-X008-Y001,EU_300-X017-Y002,EU_150-X035-Y004,4,8,17,35,0,1,2,4


The grid always contains the smallest tile scheme that the user requested and which defines the geometries of the tiles (here: `ID150`), as well as the larger schemes in which it is nested (here: `ID300`, `ID600`, `ID1200`). The X and Y position (e.g. `X035-Y003`) combined with the tile size information (e.g. `EU_150`) creates a unique and spatially explicit identifier (`EU_150-X035-Y003`).  

----

## Running geeo processing for grid scheme

Suppose we wanted to process some tiles of the EU Glance grid:

In [10]:
tile_list_to_process = ['EU_150-X029-Y029', 'EU_150-X028-Y029', 'EU_150-X028-Y030', 'EU_150-X029-Y030']

We simply specify our path to the vector file or the GeoDataFrame (only in interactive mode) while also setting `ROI_TILES: True`. Furthermore we have to specify the attribute column that contains a string name which will be appended to each output (here: `ID150`). Since we only want four tiles, we also specify the list of names found in `ROI_TILES_ATTRIBUTE_COLUMN` to `ROI_TILES_ATTRIBUTE_LIST`:

In [11]:
prm = {
    'ROI': glance_eu,
    'ROI_TILES': True,
    'ROI_TILES_ATTRIBUTE_COLUMN': 'ID150',
    'ROI_TILES_ATTRIBUTE_LIST': tile_list_to_process,
    'STM': ['p50'],
    'EXPORT_IMAGE': False,
    'EXPORT_STM': True,
    'EXPORT_DESC': 'GRID_EXAMPLE'
}
run_prm = geeo.run_param(prm)

---------------------------------------------------------
            USING ROI TILE PROCESSING MODE

ROI_TILES: ROI contains 4 features.


100%|██████████| 4/4 [00:03<00:00,  1.04it/s]

---------------------------------------------------------


In [12]:
run_prm.keys()

dict_keys(['EU_150-X028-Y029', 'EU_150-X029-Y029', 'EU_150-X028-Y030', 'EU_150-X029-Y030'])

Using the tiling processing mode or not, geeo will try and fit the desired pixel size exactly onto the given ROI (unless CRS_TRANSFORM is provided explicitly). See the above figure for a graphical illustration of an aligned pixel grid.
If the user works with a tiling scheme like the ones we constructed above, it is fairly easy to align new exports to a desired grid. See the next tutorial where we briefly illustrate custom image / image collection exports using such as alignment.

----